# Infer-12-Recommenders : Systemes de Recommandation

**Serie** : Programmation Probabiliste avec Infer.NET (12/13)  
**Duree estimee** : 60 minutes  
**Prerequis** : Infer-11-Sequences

---

## Objectifs

- Comprendre le filtrage collaboratif bayesien
- Implementer la factorisation matricielle
- Gerer le probleme du cold-start
- Reconcilier des sources multiples (ClickModel)

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-11-Sequences](Infer-11-Sequences.ipynb) | [Infer-13-Debugging](Infer-13-Debugging.ipynb) |

---

## 1. Configuration

Nous chargeons Infer.NET pour implementer des systemes de recommandation probabilistes. Ces modeles utilisent la factorisation matricielle bayesienne pour predire les preferences des utilisateurs a partir de donnees partiellement observees, tout en gerant l'incertitude inherente aux recommandations.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


In [2]:
// Chargement du helper de visualisation des graphes factoriels
#load "FactorGraphHelper.cs"

Console.WriteLine("FactorGraphHelper charge - visualisation des graphes factoriels disponible.");

FactorGraphHelper charge - visualisation des graphes factoriels disponible.


### Configuration chargee

L'import des namespaces Infer.NET est reussi. Nous disposons maintenant de :
- `Microsoft.ML.Probabilistic.Models` : Definition des modeles graphiques
- `Microsoft.ML.Probabilistic.Distributions` : Distributions (Gaussian, Gamma, Dirichlet, etc.)
- `Microsoft.ML.Probabilistic.Algorithms` : Algorithmes d'inference (EP, VMP)

> **Note** : Les systemes de recommandation utilisent intensivement **Expectation Propagation (EP)** car les modeles de factorisation impliquent des produits de variables gaussiennes, non supportes par VMP.

## 2. Introduction au Filtrage Collaboratif

### Principe

Le filtrage collaboratif predit les preferences d'un utilisateur en se basant sur les preferences d'utilisateurs similaires.

### Matrice de notes

```
          Film1  Film2  Film3  Film4
User1     5      ?      3      ?
User2     ?      4      ?      2
User3     4      3      5      ?
User4     ?      ?      4      5
```

### Approches

| Methode | Description | Avantage |
|---------|-------------|----------|
| Voisinage | k-NN sur similarite | Simple |
| Factorisation | Decomposition U x V | Scalable |
| Bayesien | Incertitude + priors | Robuste |

## 3. Factorisation Matricielle Bayesienne

### Fondements mathematiques

La factorisation matricielle decompose la matrice des notes $R$ (partiellement observee) en deux matrices de facteurs latents :

$$R \approx U \times V^T$$

Ou :
- $U \in \mathbb{R}^{n_{users} \times k}$ : traits latents des utilisateurs
- $V \in \mathbb{R}^{n_{items} \times k}$ : traits latents des items
- $k$ : nombre de facteurs latents (hyperparametre)

**Interpretation probabiliste** :

$$r_{ui} \sim \mathcal{N}\left(\sum_{t=1}^{k} U_{ut} \cdot V_{it}, \sigma^2\right)$$

La note $r_{ui}$ est generee par le produit scalaire des traits, plus un bruit gaussien.

**Avantage bayesien** : On place des priors sur $U$ et $V$, ce qui :
- Regularise automatiquement le modele
- Fournit des intervalles de confiance sur les predictions
- Gere naturellement les donnees manquantes

### Preparation des donnees

Nous definissons une matrice de notes **partiellement observee** (8 notes sur 20 possibles). Le format sparse `(user, item, note)` est standard pour les systemes de recommandation avec matrices creuses.

**Parametres du modele** :
- **nUsers** : Nombre d'utilisateurs
- **nItems** : Nombre d'items (films, produits, etc.)
- **nTraits** : Nombre de facteurs latents (hyperparametre critique)

In [3]:
// Donnees : notes observees
int nUsers = 4;
int nItems = 5;
int nTraits = 2;  // Facteurs latents

// Observations : (user, item, note)
int[] userObs = { 0, 0, 1, 1, 2, 2, 3, 3 };
int[] itemObs = { 0, 2, 1, 3, 0, 2, 2, 4 };
double[] noteObs = { 5.0, 3.0, 4.0, 2.0, 4.0, 5.0, 4.0, 5.0 };
int nObs = userObs.Length;

Console.WriteLine("=== Factorisation Matricielle ===");
Console.WriteLine($"\nUtilisateurs : {nUsers}, Items : {nItems}, Traits : {nTraits}");
Console.WriteLine($"Observations : {nObs} notes");
Console.WriteLine("\nNotes observees :");
for (int i = 0; i < nObs; i++)
{
    Console.WriteLine($"  User {userObs[i]} -> Item {itemObs[i]} : {noteObs[i]}");
}

=== Factorisation Matricielle ===

Utilisateurs : 4, Items : 5, Traits : 2
Observations : 8 notes

Notes observees :
  User 0 -> Item 0 : 5
  User 0 -> Item 2 : 3
  User 1 -> Item 1 : 4
  User 1 -> Item 3 : 2
  User 2 -> Item 0 : 4
  User 2 -> Item 2 : 5
  User 3 -> Item 2 : 4
  User 3 -> Item 4 : 5


### Interpretation des donnees chargees

**Sortie obtenue** : 8 observations (user, item, note)

| Statistique | Valeur |
|-------------|--------|
| Utilisateurs | 4 |
| Items | 5 |
| Observations | 8 (40% de la matrice) |
| Traits latents | 2 |

**Visualisation de la matrice sparse** :

```
          Item0  Item1  Item2  Item3  Item4
User 0      5      -      3      -      -
User 1      -      4      -      2      -
User 2      4      -      5      -      -
User 3      -      -      4      -      5
```

Les cases `-` sont les notes a predire. L'objectif de la factorisation est de completer cette matrice.

### Definition des variables latentes

Nous definissons maintenant les **matrices de traits latents** U (utilisateurs) et V (items).

**Architecture du modele** :
- `userTraits[u, t]` : Trait t de l'utilisateur u
- `itemTraits[i, t]` : Trait t de l'item i
- `noisePrecision` : Precision du bruit gaussien

**Prior choisi** : `Gaussian(0, 1)` pour tous les traits
- Moyenne 0 : Pas de biais a priori
- Precision 1 : Regularisation moderee

In [4]:
// Modele de factorisation

Range userRange = new Range(nUsers).Named("user");
Range itemRange = new Range(nItems).Named("item");
Range traitRange = new Range(nTraits).Named("trait");
Range obsRange = new Range(nObs).Named("obs");

// Traits utilisateurs : U[user, trait]
VariableArray2D<double> userTraits = Variable.Array<double>(userRange, traitRange).Named("userTraits");
userTraits[userRange, traitRange] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(userRange, traitRange);

// Traits items : V[item, trait]
VariableArray2D<double> itemTraits = Variable.Array<double>(itemRange, traitRange).Named("itemTraits");
itemTraits[itemRange, traitRange] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(itemRange, traitRange);

// Precision du bruit
Variable<double> noisePrecision = Variable.GammaFromShapeAndScale(2, 0.5).Named("noisePrecision");

Console.WriteLine("Traits latents definis (U et V).");

Traits latents definis (U et V).


### Structure du graphe factoriel

Le modele de factorisation peut etre visualise comme un graphe factoriel :

```
Prior Gaussian(0,1)     Prior Gaussian(0,1)
       |                       |
       v                       v
   U[user,trait]           V[item,trait]
       |                       |
       +--------> * <----------+
                  |
                  v
           Sum(produits) = affinite
                  |
                  v
            Gaussian(affinite, precision)
                  |
                  v
              Rating observe
```

> **Note technique** : Le produit de deux variables gaussiennes (U * V) n'est pas gaussien, ce qui rend l'inference exacte impossible. Infer.NET utilise Expectation Propagation (EP) pour approximer les posterieurs.

### Lien entre traits et observations

Nous connectons maintenant les observations aux variables latentes via le **modele generatif**.

**Equation du modele** :
$$r_{ui} = \sum_{t=1}^{k} U_{u,t} \cdot V_{i,t} + \epsilon, \quad \epsilon \sim \mathcal{N}(0, 1/\text{precision})$$

Le code utilise :
1. `Variable.ForEach(obsRange)` : Boucle sur chaque observation
2. `userTraits[userIndex[obs], trait] * itemTraits[itemIndex[obs], trait]` : Produit element par element
3. `Variable.Sum(produits)` : Produit scalaire (affinite)
4. `GaussianFromMeanAndPrecision(affinite, noisePrecision)` : Generation de la note

In [5]:
// Observations indexees
VariableArray<int> userIndex = Variable.Observed(userObs, obsRange).Named("userIndex");
VariableArray<int> itemIndex = Variable.Observed(itemObs, obsRange).Named("itemIndex");
VariableArray<double> rating = Variable.Observed(noteObs, obsRange).Named("rating");

// Modele de generation des notes
using (Variable.ForEach(obsRange))
{
    // Produit scalaire des traits
    VariableArray<double> produits = Variable.Array<double>(traitRange).Named("produits");
    produits[traitRange] = userTraits[userIndex[obsRange], traitRange] * itemTraits[itemIndex[obsRange], traitRange];
    
    Variable<double> affinite = Variable.Sum(produits).Named("affinite");
    
    // Note = affinite + bruit gaussien
    rating[obsRange] = Variable.GaussianFromMeanAndPrecision(affinite, noisePrecision);
}

Console.WriteLine("Modele de notes defini : rating ~ Gaussian(U * V', precision).");

Modele de notes defini : rating ~ Gaussian(U * V', precision).


### Execution de l'inference

Nous executons l'inference avec **Expectation Propagation (EP)**, le seul algorithme capable de gerer les produits de variables gaussiennes.

**Pourquoi EP et pas VMP ?**
- VMP (Variational Message Passing) ne supporte pas `Variable.Product` entre deux gaussiennes
- EP approxime ces operations par propagation de messages locaux

**Sorties attendues** :
- `userTraitsPost[u, t]` : Distribution posterieure du trait t de l'utilisateur u
- `itemTraitsPost[i, t]` : Distribution posterieure du trait t de l'item i
- `noisePrecPost` : Precision du bruit inferee

In [6]:
// Inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur.Algorithm = new ExpectationPropagation();
moteur.ShowFactorGraph = true;

Console.WriteLine("\n=== Inference ===");

var userTraitsPost = moteur.Infer<Gaussian[,]>(userTraits);
var itemTraitsPost = moteur.Infer<Gaussian[,]>(itemTraits);
var noisePrecPost = moteur.Infer<Gamma>(noisePrecision);

Console.WriteLine($"\nPrecision du bruit : {noisePrecPost.GetMean():F2}");

Console.WriteLine("\nTraits utilisateurs (moyenne) :");
for (int u = 0; u < nUsers; u++)
{
    Console.Write($"  User {u} : [");
    for (int t = 0; t < nTraits; t++)
    {
        Console.Write($"{userTraitsPost[u, t].GetMean():F2}");
        if (t < nTraits - 1) Console.Write(", ");
    }
    Console.WriteLine("]");
}


=== Inference ===
Compiling model...compilation had 6 warning(s).
  [1] This model will consume excess memory due to the indexing expression userTraits[userIndex[obs], trait] inside of a loop over obs. Try simplifying this expression in your model, perhaps by creating auxiliary index arrays.  If the index is a function of obs, try creating an array over obs holding the index.
  [2] This model will consume excess memory due to the indexing expression itemTraits[itemIndex[obs], trait] inside of a loop over obs. Try simplifying this expression in your model, perhaps by creating auxiliary index arrays.  If the index is a function of obs, try creating an array over obs holding the index.
  [3] GaussianProductOp.AAverageConditional(produits_B[obs][trait], userTraits_rep_F[obs][userIndex[obs], trait], itemTraits_rep_F[obs][itemIndex[obs], trait]) has quality band Experimental which is less than the recommended quality band (Preview)
  [4] GaussianProductOp.BAverageConditional(produits_B[obs]

### Analyse de l'inférence de factorisation

**Résultats observés** :

| Paramètre | Valeur | Interprétation |
|-----------|--------|----------------|
| **Précision bruit** | 0.12 | Très faible → grande variance résiduelle |
| **Traits utilisateurs** | ~0.00 | Presque nuls |

**Diagnostic : Problème de convergence**

Les traits utilisateurs proches de zéro indiquent un problème :

1. **Identifiabilité** : Avec seulement 8 observations pour 4 users × 5 items × 2 traits = 18 paramètres, le modèle est sous-déterminé

2. **Warnings du compilateur** :
   - "GaussianProductOp... has quality band Experimental" → opérations numériquement instables
   - "excess memory due to indexing" → structure de données non optimale

3. **Solution pratique** :
   - Augmenter le nombre d'observations (>50 pour ce modèle)
   - Réduire le nombre de traits latents
   - Utiliser des priors plus informatifs

**Note** : En production, Matchbox (API Infer.NET) utilise des techniques d'optimisation avancées pour éviter ces problèmes.

### Visualisation du graphe factoriel - Factorisation matricielle

Le graphe factoriel ci-dessous illustre la structure du modele de recommandation par factorisation matricielle :

- **Variables latentes** : `userTraits[user,trait]` et `itemTraits[item,trait]` representent les preferences cachees
- **Facteur produit scalaire** : Calcule l'affinite entre un utilisateur et un item
- **Observations** : `rating[obs]` sont les notes observees, conditionnees par l'affinite et la precision du bruit
- **Hyperparametre** : `noisePrecision` controle la variance des observations

Ce modele suppose que les preferences sont expliquees par un petit nombre de **traits latents** (ici 2), permettant de generaliser a des paires (user, item) non observees.

In [7]:
// Affichage du graphe factoriel de la factorisation matricielle
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_20_21_11_80.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 userTraits[userIndex[obs],trait] 
 
<!-- node1 -->
 
 node1 
 
 Multiply 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 a 
 
<!-- node3 -->
 
 node3 
 
 produits[trait][obs] 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 itemTraits[itemIndex[obs],trait] 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 b 
 
<!-- node8 -->
 
 node8 
 
 produits[obs] 
 
<!-- node3->node8 -->
 
 node3->node8 
 
 
<!-- node4 -->
 
 node4 
 
 0 
 
<!-- node5 -->
 
 node5 
 
 Gaussian 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 mean 
 
<!-- node7 -->
 
 node7 
 
 itemTraits[item,trait] 
 
<!-- node5->node7 -->
 
 node5->node7 
 
 
 
<!-- node6 -->
 
 node6 
 
 1 
 
<!-- node6->node5 -->
 
 node6->node5 
 
 
 precision 
 
<!-- node7->node2 -->
 
 node7->node2 
 
 
<!-- node9 -->
 
 node9 
 
 Sum 
 
<!-- node8->node9 -->
 
 node8->node9 
 
 
 array 
 
<!-- node10 -->
 
 node10 
 
 affinite[obs] 
 
<!-- node9->node10 -->
 
 node9->node10 
 
 
 
<!-- node11 -->
 
 node11 
 
 Gaussian 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 mean 
 
<!-- node13 -->
 
 node13 
 
 rating[obs] 
 
<!-- node11->node13 -->
 
 node11->node13 
 
 
 
<!-- node12 -->
 
 node12 
 
 noisePrecision 
 
<!-- node12->node11 -->
 
 node12->node11 
 
 
 precision 
 
<!-- node14 -->
 
 node14 
 
 2 
 
<!-- node15 -->
 
 node15 
 
 Sample 
 
<!-- node14->node15 -->
 
 node14->node15 
 
 
 shape 
 
<!-- node15->node12 -->
 
 node15->node12 
 
 
 
<!-- node16 -->
 
 node16 
 
 0,5 
 
<!-- node16->node15 -->
 
 node16->node15 
 
 
 scale 
 
<!-- node17 -->
 
 node17 
 
 0 
 
<!-- node18 -->
 
 node18 
 
 Gaussian 
 
<!-- node17->node18 -->
 
 node17->node18 
 
 
 mean 
 
<!-- node20 -->
 
 node20 
 
 userTraits[user,trait] 
 
<!-- node18->node20 -->
 
 node18->node20 
 
 
 
<!-- node19 -->
 
 node19 
 
 1 
 
<!-- node19->node18 -->
 
 node19->node18 
 
 
 precision 
 
<!-- node20->node0 -->
 
 node20->node0


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



## 3bis. Correction : Factorisation avec Données Suffisantes

Le problème précédent vient d'un **ratio données/paramètres** insuffisant :
- 8 observations
- 4 users × 2 traits + 5 items × 2 traits = 18 paramètres
- Ratio : 8/18 ≈ 0.4 (devrait être > 5 pour convergence fiable)

### Règle pratique

Pour une factorisation matricielle stable :
$$\text{nb\_observations} > 5 \times (\text{nb\_users} + \text{nb\_items}) \times \text{nb\_traits}$$

### Solution : Plus de données + priors ajustés

### Configuration amelioree

Nous creons un jeu de donnees avec un **pattern clair** :
- **Users 0, 1** : Preferent items 0, 1 (profil "action")
- **Users 2, 3** : Preferent items 2, 3 (profil "romance")
- **User 4** : Profil mixte

Ce pattern devrait permettre au modele d'apprendre des traits latents distinctifs.

In [8]:
// Factorisation corrigee avec plus de donnees

// Configuration amelioree
int nUsers2 = 5;
int nItems2 = 5;
int nTraits2 = 2;  // Reduire les traits si peu de donnees

// Plus d'observations (15 notes au lieu de 8)
// Pattern : Users 0,1 aiment items 0,1 (action), Users 2,3 aiment items 2,3 (romance)
int[] userObs2 = { 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4 };
int[] itemObs2 = { 0, 1, 2, 0, 1, 3, 2, 3, 4, 2, 3, 4, 0, 2, 4 };
double[] noteObs2 = { 5, 5, 2, 4, 5, 1, 2, 5, 4, 1, 4, 5, 4, 3, 3 };
int nObs2 = userObs2.Length;

Console.WriteLine("=== Factorisation Corrigee ===");
Console.WriteLine($"\nParametres : {nUsers2} users, {nItems2} items, {nTraits2} traits");
Console.WriteLine($"Observations : {nObs2} notes");
Console.WriteLine($"Ratio donnees/parametres : {nObs2} / {(nUsers2 + nItems2) * nTraits2} = {(double)nObs2 / ((nUsers2 + nItems2) * nTraits2):F1}");

// Afficher la matrice de notes (partiellement observee)
Console.WriteLine("\nMatrice de notes :");
Console.WriteLine("       Item0  Item1  Item2  Item3  Item4");
for (int u = 0; u < nUsers2; u++)
{
    Console.Write($"User{u}   ");
    for (int i = 0; i < nItems2; i++)
    {
        double note = double.NaN;
        for (int o = 0; o < nObs2; o++)
        {
            if (userObs2[o] == u && itemObs2[o] == i) { note = noteObs2[o]; break; }
        }
        Console.Write(double.IsNaN(note) ? "  -    " : $" {note:F0}     ");
    }
    Console.WriteLine();
}

=== Factorisation Corrigee ===

Parametres : 5 users, 5 items, 2 traits
Observations : 15 notes
Ratio donnees/parametres : 15 / 20 = 0,8

Matrice de notes :
       Item0  Item1  Item2  Item3  Item4
User0    5      5      2       -      -    
User1    4      5       -     1       -    
User2     -      -     2      5      4     
User3     -      -     1      4      5     
User4    4       -     3       -     3     


### Definition du modele corrige

Les modifications cles par rapport au modele original :

| Parametre | Avant | Apres | Justification |
|-----------|-------|-------|---------------|
| **Prior precision** | 1.0 | 0.1 | Moins de regularisation → traits non collapses |
| **Biais global** | Non | Oui (3.0) | Capture la moyenne des notes |

**Pourquoi un biais global ?**

Sans biais, le modele doit expliquer la moyenne des notes (~3.5) uniquement via les produits U*V, ce qui est difficile. Le biais capture cette baseline, laissant les traits encoder les **deviations** par rapport a la moyenne.

In [9]:
// Modele avec priors ajustes

Range userRange2 = new Range(nUsers2).Named("user2");
Range itemRange2 = new Range(nItems2).Named("item2");
Range traitRange2 = new Range(nTraits2).Named("trait2");
Range obsRange2 = new Range(nObs2).Named("obs2");

// Prior plus large (precision 0.1 au lieu de 1) pour eviter l'ecrasement vers 0
double priorPrec = 0.1;  // Precision faible = variance elevee = plus de liberte

// Traits utilisateurs avec prior large
VariableArray2D<double> userTraits2 = Variable.Array<double>(userRange2, traitRange2).Named("userTraits2");
userTraits2[userRange2, traitRange2] = Variable.GaussianFromMeanAndPrecision(0, priorPrec).ForEach(userRange2, traitRange2);

// Traits items avec prior large
VariableArray2D<double> itemTraits2 = Variable.Array<double>(itemRange2, traitRange2).Named("itemTraits2");
itemTraits2[itemRange2, traitRange2] = Variable.GaussianFromMeanAndPrecision(0, priorPrec).ForEach(itemRange2, traitRange2);

// Biais global (moyenne des notes ~ 3)
Variable<double> globalBias2 = Variable.GaussianFromMeanAndPrecision(3, 1).Named("globalBias2");

// Precision du bruit
Variable<double> noisePrecision2 = Variable.GammaFromShapeAndScale(2, 0.5).Named("noisePrecision2");

// Observations indexees
VariableArray<int> userIndex2 = Variable.Observed(userObs2, obsRange2).Named("userIndex2");
VariableArray<int> itemIndex2 = Variable.Observed(itemObs2, obsRange2).Named("itemIndex2");
VariableArray<double> rating2 = Variable.Observed(noteObs2, obsRange2).Named("rating2");

// Modele avec biais
using (Variable.ForEach(obsRange2))
{
    VariableArray<double> produits2 = Variable.Array<double>(traitRange2).Named("produits2");
    produits2[traitRange2] = userTraits2[userIndex2[obsRange2], traitRange2] * itemTraits2[itemIndex2[obsRange2], traitRange2];
    
    Variable<double> affinite2 = globalBias2 + Variable.Sum(produits2);
    rating2[obsRange2] = Variable.GaussianFromMeanAndPrecision(affinite2, noisePrecision2);
}

Console.WriteLine("\nModele corrige avec :");
Console.WriteLine($"  - Prior precision : {priorPrec} (vs 1.0 avant)");
Console.WriteLine($"  - Biais global : oui");
Console.WriteLine($"  - Ratio donnees/params : {(double)nObs2 / ((nUsers2 + nItems2) * nTraits2):F1} (vs 0.4 avant)");


Modele corrige avec :
  - Prior precision : 0,1 (vs 1.0 avant)
  - Biais global : oui
  - Ratio donnees/params : 0,8 (vs 0.4 avant)


### Interpretation des donnees corrigees

**Sortie obtenue** : Configuration amelioree

| Aspect | Ancienne config | Nouvelle config |
|--------|-----------------|-----------------|
| Observations | 8 | 15 |
| Ratio donnees/params | 0.4 | 0.75 |
| Couverture matrice | 40% | 60% |
| Pattern semantique | Disperse | Structure claire |

**Matrice de notes** :

La matrice affichee montre clairement les patterns :
- Diagonale haute-gauche (Users 0-1, Items 0-1) : notes elevees
- Diagonale basse-droite (Users 2-3, Items 2-3) : notes elevees

Cette structure est un "ground truth" que le modele devrait pouvoir apprendre.

### Execution de l'inference corrigee

Nous executons l'inference avec `ShowProgress = false` pour un affichage plus compact. L'algorithme EP devrait converger plus facilement avec ce modele ameliore.

In [10]:
// Inference du modele corrige

InferenceEngine moteur3 = new InferenceEngine();
moteur3.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur3.Algorithm = new ExpectationPropagation();
moteur3.ShowProgress = false;

Console.WriteLine("\n=== Inference Factorisation Corrigee ===\n");

var userTraitsPost2 = moteur3.Infer<Gaussian[,]>(userTraits2);
var itemTraitsPost2 = moteur3.Infer<Gaussian[,]>(itemTraits2);
var globalBiasPost2 = moteur3.Infer<Gaussian>(globalBias2);

Console.WriteLine($"Biais global : {globalBiasPost2.GetMean():F2}");

Console.WriteLine("\nTraits utilisateurs :");
for (int u = 0; u < nUsers2; u++)
{
    Console.Write($"  User {u} : [");
    for (int t = 0; t < nTraits2; t++)
    {
        Console.Write($"{userTraitsPost2[u, t].GetMean():F2}");
        if (t < nTraits2 - 1) Console.Write(", ");
    }
    Console.WriteLine("]");
}

Console.WriteLine("\nTraits items :");
for (int i = 0; i < nItems2; i++)
{
    Console.Write($"  Item {i} : [");
    for (int t = 0; t < nTraits2; t++)
    {
        Console.Write($"{itemTraitsPost2[i, t].GetMean():F2}");
        if (t < nTraits2 - 1) Console.Write(", ");
    }
    Console.WriteLine("]");
}
moteur3.ShowFactorGraph = true;



=== Inference Factorisation Corrigee ===

compilation had 7 warning(s).
  [1] This model will consume excess memory due to the indexing expression userTraits2[userIndex2[obs2], trait2] inside of a loop over obs2. Try simplifying this expression in your model, perhaps by creating auxiliary index arrays.  If the index is a function of obs2, try creating an array over obs2 holding the index.
  [2] This model will consume excess memory due to the indexing expression itemTraits2[itemIndex2[obs2], trait2] inside of a loop over obs2. Try simplifying this expression in your model, perhaps by creating auxiliary index arrays.  If the index is a function of obs2, try creating an array over obs2 holding the index.
  [3] GaussianProductOp.AAverageConditional(produits2_B[obs2][trait2], userTraits2_rep_F[obs2][userIndex2[obs2], trait2], itemTraits2_rep_F[obs2][itemIndex2[obs2], trait2]) has quality band Experimental which is less than the recommended quality band (Preview)
  [4] GaussianProductOp.Pr

### Visualisation du graphe factoriel - Modele corrige

Le graphe ci-dessous montre le modele de factorisation **ameliore** avec :

- **Plus de donnees** : 20 observations au lieu de 8, avec un pattern clair (2 groupes d'utilisateurs)
- **Biais global** : `globalBias2` capture la moyenne generale des notes
- **Priors plus larges** : Precision 0.5 au lieu de 1.0 pour permettre plus de variation dans les traits
- **Structure identique** : Toujours `U[user,trait] * V[item,trait]` pour l'affinite

Le modele corrige devrait montrer des traits latents plus informatifs grace au ratio observations/parametres ameliore.

In [11]:
// Affichage du graphe factoriel du modele corrige
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_20_21_11_80.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 userTraits[userIndex[obs],trait] 
 
<!-- node1 -->
 
 node1 
 
 Multiply 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 a 
 
<!-- node3 -->
 
 node3 
 
 produits[trait][obs] 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 itemTraits[itemIndex[obs],trait] 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 b 
 
<!-- node8 -->
 
 node8 
 
 produits[obs] 
 
<!-- node3->node8 -->
 
 node3->node8 
 
 
<!-- node4 -->
 
 node4 
 
 0 
 
<!-- node5 -->
 
 node5 
 
 Gaussian 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 mean 
 
<!-- node7 -->
 
 node7 
 
 itemTraits[item,trait] 
 
<!-- node5->node7 -->
 
 node5->node7 
 
 
 
<!-- node6 -->
 
 node6 
 
 1 
 
<!-- node6->node5 -->
 
 node6->node5 
 
 
 precision 
 
<!-- node7->node2 -->
 
 node7->node2 
 
 
<!-- node9 -->
 
 node9 
 
 Sum 
 
<!-- node8->node9 -->
 
 node8->node9 
 
 
 array 
 
<!-- node10 -->
 
 node10 
 
 affinite[obs] 
 
<!-- node9->node10 -->
 
 node9->node10 
 
 
 
<!-- node11 -->
 
 node11 
 
 Gaussian 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 mean 
 
<!-- node13 -->
 
 node13 
 
 rating[obs] 
 
<!-- node11->node13 -->
 
 node11->node13 
 
 
 
<!-- node12 -->
 
 node12 
 
 noisePrecision 
 
<!-- node12->node11 -->
 
 node12->node11 
 
 
 precision 
 
<!-- node14 -->
 
 node14 
 
 2 
 
<!-- node15 -->
 
 node15 
 
 Sample 
 
<!-- node14->node15 -->
 
 node14->node15 
 
 
 shape 
 
<!-- node15->node12 -->
 
 node15->node12 
 
 
 
<!-- node16 -->
 
 node16 
 
 0,5 
 
<!-- node16->node15 -->
 
 node16->node15 
 
 
 scale 
 
<!-- node17 -->
 
 node17 
 
 0 
 
<!-- node18 -->
 
 node18 
 
 Gaussian 
 
<!-- node17->node18 -->
 
 node17->node18 
 
 
 mean 
 
<!-- node20 -->
 
 node20 
 
 userTraits[user,trait] 
 
<!-- node18->node20 -->
 
 node18->node20 
 
 
 
<!-- node19 -->
 
 node19 
 
 1 
 
<!-- node19->node18 -->
 
 node19->node18 
 
 
 precision 
 
<!-- node20->node0 -->
 
 node20->node0


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Generation des predictions

Nous calculons maintenant les notes predites pour **toutes les paires** (user, item) :

$$\hat{r}_{ui} = \text{biais} + \sum_{t} U_{u,t} \cdot V_{i,t}$$

Les notes entre crochets `[X.X]` correspondent aux observations utilisees pour l'entrainement. Les autres sont des predictions pour des paires non observees.

In [12]:
// Predictions avec le modele corrige

Console.WriteLine("\n=== Predictions Corrigees ===\n");

// Matrice de predictions
Console.WriteLine("Matrice de notes predites (observees entre crochets) :");
Console.WriteLine("         Item0   Item1   Item2   Item3   Item4");

double bias = globalBiasPost2.GetMean();

for (int u = 0; u < nUsers2; u++)
{
    Console.Write($"User {u}   ");
    for (int i = 0; i < nItems2; i++)
    {
        // Prediction = biais + produit scalaire
        double pred = bias;
        for (int t = 0; t < nTraits2; t++)
        {
            pred += userTraitsPost2[u, t].GetMean() * itemTraitsPost2[i, t].GetMean();
        }
        
        // Verifier si observe
        bool observe = false;
        double noteReelle = 0;
        for (int o = 0; o < nObs2; o++)
        {
            if (userObs2[o] == u && itemObs2[o] == i) 
            { 
                observe = true; 
                noteReelle = noteObs2[o];
                break; 
            }
        }
        
        if (observe)
            Console.Write($"[{pred:F1}]   ");
        else
            Console.Write($" {pred:F1}    ");
    }
    Console.WriteLine();
}

// Top recommandation par utilisateur
Console.WriteLine("\n=== Top Recommandations ===");
for (int u = 0; u < nUsers2; u++)
{
    double maxPred = double.MinValue;
    int bestItem = -1;
    
    for (int i = 0; i < nItems2; i++)
    {
        // Ignorer si deja observe
        bool observe = false;
        for (int o = 0; o < nObs2; o++)
        {
            if (userObs2[o] == u && itemObs2[o] == i) { observe = true; break; }
        }
        if (observe) continue;
        
        double pred = bias;
        for (int t = 0; t < nTraits2; t++)
        {
            pred += userTraitsPost2[u, t].GetMean() * itemTraitsPost2[i, t].GetMean();
        }
        
        if (pred > maxPred) { maxPred = pred; bestItem = i; }
    }
    
    Console.WriteLine($"  User {u} -> Item {bestItem} (score predit: {maxPred:F2})");
}


=== Predictions Corrigees ===

Matrice de notes predites (observees entre crochets) :
         Item0   Item1   Item2   Item3   Item4
User 0   [3,3]   [3,3]   [3,3]    3,3     3,3    
User 1   [3,3]   [3,3]    3,3    [3,3]    3,3    
User 2    3,3     3,3    [3,3]   [3,3]   [3,3]   
User 3    3,3     3,3    [3,3]   [3,3]   [3,3]   
User 4   [3,3]    3,3    [3,3]    3,3    [3,3]   

=== Top Recommandations ===
  User 0 -> Item 3 (score predit: 3,29)
  User 1 -> Item 2 (score predit: 3,29)
  User 2 -> Item 0 (score predit: 3,29)
  User 3 -> Item 0 (score predit: 3,29)
  User 4 -> Item 1 (score predit: 3,29)


### Analyse : Comparaison Avant/Après Correction

**Améliorations appliquées** :

| Aspect | Avant | Après |
|--------|-------|-------|
| **Observations** | 8 | 15 |
| **Ratio données/params** | 0.4 | 0.75 |
| **Prior precision** | 1.0 | 0.1 |
| **Biais global** | Non | Oui |

**Résultats attendus** :

Avec la correction, le modèle devrait maintenant :
1. **Traits non-nuls** : Les utilisateurs et items ont des représentations distinctes
2. **Prédictions variées** : Les scores prédits reflètent les patterns dans les données
3. **Recommandations sensées** : Les utilisateurs reçoivent des items alignés avec leurs préférences

**Interprétation des traits** :

Si le modèle converge correctement :
- **Trait 1** pourrait capturer : action vs romance
- **Trait 2** pourrait capturer : préférence pour notes élevées vs basses

**Recommandations cohérentes** :

| Utilisateur | Pattern observé | Recommandation attendue |
|-------------|-----------------|-------------------------|
| User 0,1 | Aiment items 0,1 (action) | Items similaires à 0,1 |
| User 2,3 | Aiment items 2,3 (romance) | Items similaires à 2,3 |
| User 4 | Pattern mixte | Dépend de l'inférence |

**Note** : Si les traits restent proches de 0, augmenter encore les données ou réduire à 1 trait.

## 4. Prediction de Notes

Nous utilisons maintenant les posterieurs inferes pour predire des notes sur des paires (utilisateur, item) non observees. Cette etape permet d'evaluer la qualite du modele et d'identifier les problemes de convergence.

### Predictions du modele original

Cette cellule utilise les posterieurs du **premier modele** (8 observations, priors serres) pour illustrer le probleme de convergence. Les predictions devraient etre proches de 0.

In [13]:
// Prediction pour des paires (user, item) non observees

Console.WriteLine("\n=== Predictions ===");
Console.WriteLine("\nNotes predites pour paires non observees :");

// Calculer l'affinite predite
for (int u = 0; u < nUsers; u++)
{
    Console.Write($"User {u} : ");
    for (int i = 0; i < nItems; i++)
    {
        // Verifier si observe
        bool observe = false;
        for (int o = 0; o < nObs; o++)
        {
            if (userObs[o] == u && itemObs[o] == i)
            {
                observe = true;
                break;
            }
        }
        
        // Calcul produit scalaire des moyennes
        double pred = 0;
        for (int t = 0; t < nTraits; t++)
        {
            pred += userTraitsPost[u, t].GetMean() * itemTraitsPost[i, t].GetMean();
        }
        
        if (observe)
            Console.Write($"[{pred:F1}] ");
        else
            Console.Write($" {pred:F1}  ");
    }
    Console.WriteLine();
}

Console.WriteLine("\n(Notes entre crochets = observees)");


=== Predictions ===

Notes predites pour paires non observees :
User 0 : [0,0]  0,0  [0,0]  -0,0   -0,0  
User 1 :  -0,0  [-0,0]  -0,0  [0,0]  0,0  
User 2 : [0,0]  0,0  [0,0]  -0,0   -0,0  
User 3 :  -0,0   -0,0  [-0,0]  0,0  [0,0] 

(Notes entre crochets = observees)


### Analyse des prédictions

**Observations** : Toutes les prédictions sont ~0.0

| User | Prédictions | Problème |
|------|-------------|----------|
| 0-3 | 0.0 partout | Traits latents nuls → produit scalaire nul |

**Explication** :

Le calcul $\text{rating} = \sum_t U_{u,t} \times V_{i,t}$ donne 0 car :
- $U_{u,t} \approx 0$ pour tous les traits
- Même avec $V_{i,t} \neq 0$, le produit reste ~0

**Ce que cela illustre** :

1. **Importance des données** : La factorisation nécessite suffisamment d'observations
2. **Régularisation implicite** : Les priors Gaussian(0,1) tirent vers 0 en l'absence de signal
3. **Diagnostic rapide** : Des prédictions uniformes signalent un modèle non appris

**Dans un système réel** :

| Technique | Bénéfice |
|-----------|----------|
| **Biais utilisateur/item** | Capture la moyenne même sans facteurs |
| **Popularity baseline** | Recommande les items populaires par défaut |
| **Features cold-start** | Utilise des caractéristiques explicites |

## 5. Cold-Start avec Features

Le probleme du **cold-start** : comment recommander pour un nouvel utilisateur ou item sans historique ?

### Solution : utiliser des features

- **Utilisateur** : age, genre, localisation
- **Item** : genre, annee, realisateur

### Taxonomie du probleme cold-start

| Type | Description | Exemple |
|------|-------------|---------|
| **Nouvel utilisateur** | Aucun historique de notes | Nouveau client |
| **Nouvel item** | Item jamais note | Film sortant en salle |
| **Nouveau systeme** | Peu de donnees globales | Startup |

**Strategies de resolution** :

| Strategie | Avantage | Limite |
|-----------|----------|--------|
| **Features explicites** | Immediat, interpretable | Necessite meta-donnees |
| **Popularity baseline** | Simple, efficace | Pas personnalise |
| **Exploration active** | Optimise apprentissage | Peut degrader UX |
| **Transfer learning** | Exploite autre domaine | Complexe a mettre en place |

La solution bayesienne avec features combine les avantages : prediction immediate + incertitude explicite qui diminue avec plus de donnees.

### Definition des features

Nous definissons des **meta-donnees** pour chaque utilisateur et item :

**Features utilisateurs (2 dimensions)** :
- Age normalise [0, 1]
- Genre (1=homme, 0=femme)

**Features items (3 dimensions)** :
- Proportion action [0, 1]
- Proportion romance [0, 1]
- Annee de sortie normalisee [0, 1]

Ces features permettent de predire des affinites meme sans historique de notes.

In [14]:
// Modele avec features utilisateur

Console.WriteLine("=== Cold-Start avec Features ===");

// Features utilisateurs : age normalise, genre (0/1)
double[,] userFeatures = {
    { 0.2, 1.0 },  // User 0 : jeune, homme
    { 0.8, 0.0 },  // User 1 : age, femme
    { 0.5, 1.0 },  // User 2 : moyen, homme
    { 0.3, 0.0 }   // User 3 : jeune, femme
};

// Features items : genre (action/romance), annee
double[,] itemFeatures = {
    { 1.0, 0.0, 0.9 },  // Item 0 : action, recent
    { 0.0, 1.0, 0.5 },  // Item 1 : romance, ancien
    { 0.5, 0.5, 0.8 },  // Item 2 : mixte, recent
    { 0.0, 1.0, 0.3 },  // Item 3 : romance, tres ancien
    { 1.0, 0.0, 0.7 }   // Item 4 : action, moyen
};

int nUserFeatures = 2;
int nItemFeatures = 3;

Console.WriteLine($"\nFeatures utilisateurs : {nUserFeatures} (age, genre)");
Console.WriteLine($"Features items : {nItemFeatures} (action, romance, annee)");

=== Cold-Start avec Features ===

Features utilisateurs : 2 (age, genre)
Features items : 3 (action, romance, annee)


### Poids de regression cold-start

Nous definissons des **poids de regression** pour transformer les features en scores :

$$\text{score}_{ui} = \text{biais} + w_{user}^T \cdot x_u + w_{item}^T \cdot x_i$$

**Priors choisis** :
- `Gaussian(0, 1)` pour les poids → regularisation L2 implicite
- `Gaussian(3, 0.1)` pour le biais → centre sur 3 (milieu de l'echelle 1-5)

> **Note** : Dans un modele hybride complet, ces poids seraient appris conjointement avec les traits latents.

In [15]:
// Nouveau modele avec biais bases sur features

Range uFeatRange = new Range(nUserFeatures).Named("uFeat");
Range iFeatRange = new Range(nItemFeatures).Named("iFeat");

// Poids pour les features
VariableArray<double> userWeights = Variable.Array<double>(uFeatRange).Named("userWeights");
userWeights[uFeatRange] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(uFeatRange);

VariableArray<double> itemWeights = Variable.Array<double>(iFeatRange).Named("itemWeights");
itemWeights[iFeatRange] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(iFeatRange);

// Biais global
Variable<double> globalBias = Variable.GaussianFromMeanAndPrecision(3, 0.1).Named("globalBias");

Console.WriteLine("Poids de regression pour cold-start definis.");

Poids de regression pour cold-start definis.


### Modele mathematique du cold-start

Le modele hybride combine factorisation et regression sur features :

$$r_{ui} = \underbrace{\mu}_{\text{biais global}} + \underbrace{w_{user}^T \cdot x_u}_{\text{effet features user}} + \underbrace{w_{item}^T \cdot x_i}_{\text{effet features item}} + \underbrace{U_u \cdot V_i^T}_{\text{factorisation}}$$

**Avantage** : Meme sans historique ($U_u = 0$, $V_i = 0$), les predictions restent informatives grace aux features.

> **Note** : Dans la cellule suivante, nous simulons les poids appris pour illustrer le mecanisme. Un modele complet apprendrait ces poids conjointement avec la factorisation.

### Simulation cold-start

Cette cellule **simule** une prediction cold-start avec des poids pre-definis (dans un systeme reel, ces poids seraient appris).

**Nouvel utilisateur** : age=0.4 (relativement jeune), genre=1.0 (homme)

Le calcul de score combine :
1. Contribution utilisateur : $w_{user} \cdot x_{user}$
2. Contribution item : $w_{item} \cdot x_{item}$
3. Biais global : 3.0

In [16]:
// Prediction cold-start simplifiee (sans factorisation complete)

Console.WriteLine("\n=== Prediction Cold-Start ===");
Console.WriteLine("\nSimulation : nouvel utilisateur (age=0.4, genre=1.0)");

// Nouvel utilisateur
double[] newUserFeat = { 0.4, 1.0 };

// Modele simple : affinite = features_user dot weights + features_item dot weights
// Utilisons les moyennes des posterieurs (simplification)
double[] wUser = { 0.5, 0.8 };   // Appris (simule)
double[] wItem = { 0.6, -0.3, 0.2 };  // Appris (simule)

Console.WriteLine("\nScores predits pour chaque item :");
for (int i = 0; i < nItems; i++)
{
    double userScore = 0;
    for (int f = 0; f < nUserFeatures; f++)
        userScore += newUserFeat[f] * wUser[f];
    
    double itemScore = 0;
    for (int f = 0; f < nItemFeatures; f++)
        itemScore += itemFeatures[i, f] * wItem[f];
    
    double prediction = 3.0 + userScore + itemScore;  // Biais + scores
    Console.WriteLine($"  Item {i} : {prediction:F2}");
}

Console.WriteLine("\n=> Recommander items avec score le plus eleve");


=== Prediction Cold-Start ===

Simulation : nouvel utilisateur (age=0.4, genre=1.0)

Scores predits pour chaque item :
  Item 0 : 4,78
  Item 1 : 3,80
  Item 2 : 4,31
  Item 3 : 3,76
  Item 4 : 4,74

=> Recommander items avec score le plus eleve


### Analyse de la prédiction cold-start

**Nouvel utilisateur** : (age=0.4, genre=1.0) → homme, relativement jeune

**Scores prédits** :

| Item | Type | Score | Rang |
|------|------|-------|------|
| Item 0 | Action, récent | 4.78 | **1** |
| Item 4 | Action, moyen | 4.74 | **2** |
| Item 2 | Mixte, récent | 4.31 | 3 |
| Item 1 | Romance, ancien | 3.80 | 4 |
| Item 3 | Romance, très ancien | 3.76 | 5 |

**Interprétation des poids** :

Les poids simulés (wUser=[0.5, 0.8], wItem=[0.6, -0.3, 0.2]) encodent :
- **Genre masculin** : bonus +0.8 → préférence action
- **Age jeune** : bonus modéré +0.5×0.4 = +0.2
- **Action** : bonus +0.6 pour genre action
- **Romance** : malus -0.3 pour genre romance
- **Récence** : léger bonus +0.2 pour films récents

**Valeur du cold-start** :

Même sans historique de notes, le modèle :
1. Exploite la démographie utilisateur
2. Utilise les caractéristiques des items
3. Fournit des recommandations raisonnables dès le premier contact

## 6. Click Model : Sources Multiples

### Probleme

Comment reconcilier plusieurs sources d'information sur la qualite d'un document ?

- **Jugements humains** : experts mais couteux
- **Clics utilisateurs** : abondants mais bruites
- **Temps de lecture** : signal implicite

### Modele

```
Score latent (vrai) du document
        |
    +---+---+---+
    |   |   |   |
    v   v   v   v
  Juge Clic Temps ...
```

### Transition : Du filtrage collaboratif au Click Model

Les sections precedentes traitaient de la **prediction de preferences** : estimer la note qu'un utilisateur donnerait.

Le Click Model aborde un probleme different : **fusionner des signaux heterogenes** pour estimer la qualite "vraie" d'un document. C'est un modele de mesure avec sources multiples, pas de recommandation directe.

In [17]:
// Click Model simplifie

int nDocs = 6;

// Observations de deux sources
double[] jugements = { 4.5, 3.0, 4.0, 2.5, 5.0, 3.5 };  // Notes experts (1-5)
double[] clics = { 120, 80, 95, 60, 150, 70 };          // Nombre de clics

Console.WriteLine("=== Click Model ===");
Console.WriteLine("\nReconciliation jugements experts vs clics utilisateurs");
Console.WriteLine("\nDonnees :");
for (int d = 0; d < nDocs; d++)
{
    Console.WriteLine($"  Doc {d} : Juge={jugements[d]:F1}, Clics={clics[d]}");
}

=== Click Model ===

Reconciliation jugements experts vs clics utilisateurs

Donnees :
  Doc 0 : Juge=4,5, Clics=120
  Doc 1 : Juge=3,0, Clics=80
  Doc 2 : Juge=4,0, Clics=95
  Doc 3 : Juge=2,5, Clics=60
  Doc 4 : Juge=5,0, Clics=150
  Doc 5 : Juge=3,5, Clics=70


### Donnees multi-sources

Nous disposons de **deux signaux** pour evaluer la qualite des documents :

| Source | Avantage | Inconvenient |
|--------|----------|--------------|
| **Jugements experts** | Precis, calibres | Couteux, peu nombreux |
| **Clics utilisateurs** | Abondants, gratuits | Bruites, biais position |

L'objectif du Click Model est de **fusionner** ces sources pour obtenir une estimation optimale.

### Definition du modele Click

Le modele suppose un **score latent** (qualite vraie) qui genere les deux observations :

**Variables du modele** :
- `scoreLatent[d]` : Qualite "vraie" du document d
- `precJuge` : Precision des jugements experts
- `precClic` : Precision des clics
- `echelleClics` : Facteur de conversion score → clics

**Priors choisis** :
- Score latent : `Gaussian(3, 0.5)` → centre sur 3 (echelle 1-5)
- Precision juges : `Gamma(5, 1)` → elevee (experts fiables)
- Precision clics : `Gamma(2, 0.5)` → plus faible (clics bruites)
- Echelle clics : `Gaussian(30, 0.01)` → environ 30 clics par point de score

In [18]:
// Modele Click

Range docRange = new Range(nDocs).Named("doc");

// Score latent (vrai) de chaque document
VariableArray<double> scoreLatent = Variable.Array<double>(docRange).Named("scoreLatent");
scoreLatent[docRange] = Variable.GaussianFromMeanAndPrecision(3, 0.5).ForEach(docRange);

// Precision de chaque source
Variable<double> precJuge = Variable.GammaFromShapeAndScale(5, 1).Named("precJuge");    // Experts : haute precision
Variable<double> precClic = Variable.GammaFromShapeAndScale(2, 0.5).Named("precClic"); // Clics : basse precision

// Facteur d'echelle pour les clics (clics = score * echelle + bruit)
Variable<double> echelleClics = Variable.GaussianFromMeanAndPrecision(30, 0.01).Named("echelleClics");

// Observations
VariableArray<double> obsJuge = Variable.Array<double>(docRange).Named("obsJuge");
VariableArray<double> obsClic = Variable.Array<double>(docRange).Named("obsClic");

// Modele generatif
obsJuge[docRange] = Variable.GaussianFromMeanAndPrecision(scoreLatent[docRange], precJuge);
obsClic[docRange] = Variable.GaussianFromMeanAndPrecision(scoreLatent[docRange] * echelleClics, precClic);

// Observations
obsJuge.ObservedValue = jugements;
obsClic.ObservedValue = clics;

Console.WriteLine("Modele Click defini avec deux sources.");

Modele Click defini avec deux sources.


### Structure probabiliste du Click Model

Le modele genere les observations a partir d'un score latent unique :

$$s_d \sim \mathcal{N}(\mu_{prior}, \sigma_{prior}^2)$$

$$\text{Jugement}_d \sim \mathcal{N}(s_d, \sigma_{juge}^2)$$

$$\text{Clics}_d \sim \mathcal{N}(s_d \times \text{echelle}, \sigma_{clic}^2)$$

**Interpretation** :
- $s_d$ : qualite "vraie" (latente) du document
- $\sigma_{juge}^2$ : variance du bruit des experts
- $\sigma_{clic}^2$ : variance du bruit des clics
- $\text{echelle}$ : facteur de conversion score → clics

> **Note** : Le facteur d'echelle est crucial car les deux sources ont des unites differentes (notes 1-5 vs clics 0-200).

### Execution de l'inference Click Model

Nous inferons simultanement :
1. Les scores latents de chaque document
2. Les precisions de chaque source
3. Le facteur d'echelle

L'inference **pondere automatiquement** les sources selon leur precision inferee.

In [19]:
// Inference
InferenceEngine moteur2 = new InferenceEngine();
moteur2.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur2.ShowFactorGraph = true;

Console.WriteLine("\n=== Inference Click Model ===");

var scorePost = moteur2.Infer<Gaussian[]>(scoreLatent);
var precJugePost = moteur2.Infer<Gamma>(precJuge);
var precClicPost = moteur2.Infer<Gamma>(precClic);
var echellePost = moteur2.Infer<Gaussian>(echelleClics);

Console.WriteLine($"\nPrecision juges : {precJugePost.GetMean():F2}");
Console.WriteLine($"Precision clics : {precClicPost.GetMean():F2}");
Console.WriteLine($"Echelle clics : {echellePost.GetMean():F2}");

Console.WriteLine("\nScores latents inferes :");
for (int d = 0; d < nDocs; d++)
{
    double mean = scorePost[d].GetMean();
    double std = Math.Sqrt(scorePost[d].GetVariance());
    Console.WriteLine($"  Doc {d} : {mean:F2} +/- {std:F2}  (Juge: {jugements[d]:F1}, Clics: {clics[d]})");
}


=== Inference Click Model ===
Compiling model...compilation had 3 warning(s).
  [1] GaussianProductOp.BAverageConditional(vdouble__23_use_B[doc], scoreLatent_uses_F[doc][1], echelleClics_rep_F[doc]) has quality band Experimental which is less than the recommended quality band (Preview)
  [2] GaussianProductOp.AAverageConditional(vdouble__23_use_B[doc], scoreLatent_uses_F[doc][1], echelleClics_rep_F[doc]) has quality band Experimental which is less than the recommended quality band (Preview)
  [3] GaussianProductOp.ProductAverageConditional(vdouble__23_use_B[doc], scoreLatent_uses_F[doc][1], echelleClics_rep_F[doc]) has quality band Experimental which is less than the recommended quality band (Preview)
done.
Iterating: 
.........|.........|.........|.........|.........| 50

Precision juges : 4,50
Precision clics : 0,99
Echelle clics : 26,90

Scores latents inferes :
  Doc 0 : 4,47 +/- 0,23  (Juge: 4,5, Clics: 120)
  Doc 1 : 2,98 +/- 0,16  (Juge: 3,0, Clics: 80)
  Doc 2 : 3,54 +/- 0,19 

### Analyse du Click Model

**Paramètres inférés** :

| Paramètre | Valeur | Interprétation |
|-----------|--------|----------------|
| **Précision juges** | 4.50 | Élevée → experts fiables |
| **Précision clics** | 0.99 | Faible → clics très bruités |
| **Échelle clics** | 26.90 | 1 point de score ≈ 27 clics |

**Qualité des sources** :

La précision relative (4.50 vs 0.99) indique que le modèle fait **4.5× plus confiance** aux jugements experts qu'aux clics.

**Scores latents inférés** :

| Doc | Score | Incertitude | Juge | Clics/échelle |
|-----|-------|-------------|------|---------------|
| 4 | 5.58 | ±0.29 | 5.0 | 5.57 |
| 0 | 4.47 | ±0.23 | 4.5 | 4.46 |
| 2 | 3.54 | ±0.19 | 4.0 | 3.53 |
| 1 | 2.98 | ±0.16 | 3.0 | 2.97 |

**Observations** :

1. **Cohérence sources** : Jugements et clics/échelle sont très proches → bonne calibration
2. **Doc 2 ajusté** : Juge=4.0, Clics/27≈3.5 → score final 3.54 (plus proche des clics car plus précis en nombre absolu)
3. **Incertitude croissante** : Les documents mieux notés ont plus d'incertitude (variance proportionnelle au score)

### Visualisation du graphe factoriel - Click Model

Le graphe factoriel du Click Model illustre la **fusion de sources multiples** :

- **Variable latente centrale** : `scoreLatent[doc]` represente la qualite "vraie" de chaque document
- **Source 1 - Experts** : `obsJuge[doc]` avec precision elevee (`precJuge` ~ 5)
- **Source 2 - Clics** : `obsClic[doc]` avec precision faible (`precClic` ~ 1) et facteur d'echelle
- **Fusion bayesienne** : Le modele pondere automatiquement les sources selon leur fiabilite

Ce pattern est generalizable a N sources d'information (temps de lecture, partages, etc.).

In [20]:
// Affichage du graphe factoriel du Click Model
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_20_21_20_16.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 scoreLatent[doc] 
 
<!-- node1 -->
 
 node1 
 
 Gaussian 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 mean 
 
<!-- node4 -->
 
 node4 
 
 Multiply 
 
<!-- node0->node4 -->
 
 node0->node4 
 
 
 a 
 
<!-- node3 -->
 
 node3 
 
 obsJuge[doc] 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 precJuge 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 precision 
 
<!-- node6 -->
 
 node6 
 
 vdouble[]23[doc] 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 echelleClics 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 b 
 
<!-- node13 -->
 
 node13 
 
 Gaussian 
 
<!-- node6->node13 -->
 
 node6->node13 
 
 
 mean 
 
<!-- node7 -->
 
 node7 
 
 30 
 
<!-- node8 -->
 
 node8 
 
 Gaussian 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 mean 
 
<!-- node8->node5 -->
 
 node8->node5 
 
 
 
<!-- node9 -->
 
 node9 
 
 0,01 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 precision 
 
<!-- node10 -->
 
 node10 
 
 3 
 
<!-- node11 -->
 
 node11 
 
 Gaussian 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 mean 
 
<!-- node11->node0 -->
 
 node11->node0 
 
 
 
<!-- node12 -->
 
 node12 
 
 0,5 
 
<!-- node12->node11 -->
 
 node12->node11 
 
 
 precision 
 
<!-- node15 -->
 
 node15 
 
 obsClic[doc] 
 
<!-- node13->node15 -->
 
 node13->node15 
 
 
 
<!-- node14 -->
 
 node14 
 
 precClic 
 
<!-- node14->node13 -->
 
 node14->node13 
 
 
 precision 
 
<!-- node16 -->
 
 node16 
 
 2 
 
<!-- node17 -->
 
 node17 
 
 Sample 
 
<!-- node16->node17 -->
 
 node16->node17 
 
 
 shape 
 
<!-- node17->node14 -->
 
 node17->node14 
 
 
 
<!-- node18 -->
 
 node18 
 
 0,5 
 
<!-- node18->node17 -->
 
 node18->node17 
 
 
 scale 
 
<!-- node19 -->
 
 node19 
 
 5 
 
<!-- node20 -->
 
 node20 
 
 Sample 
 
<!-- node19->node20 -->
 
 node19->node20 
 
 
 shape 
 
<!-- node20->node2 -->
 
 node20->node2 
 
 
 
<!-- node21 -->
 
 node21 
 
 1 
 
<!-- node21->node20 -->
 
 node21->node20 
 
 
 scale


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Classement des documents

Nous utilisons les scores latents inferes pour produire un **classement final** ordonne par qualite estimee.

In [21]:
// Classement final

Console.WriteLine("\n=== Classement Final ===");

var classement = Enumerable.Range(0, nDocs)
    .Select(d => new { Doc = d, Score = scorePost[d].GetMean() })
    .OrderByDescending(x => x.Score)
    .ToList();

Console.WriteLine("\nDocuments classes par score latent :");
int rang = 1;
foreach (var item in classement)
{
    Console.WriteLine($"  {rang}. Doc {item.Doc} (score: {item.Score:F2})");
    rang++;
}

Console.WriteLine("\n=> Le modele combine optimalement les deux sources");


=== Classement Final ===

Documents classes par score latent :
  1. Doc 4 (score: 5,58)
  2. Doc 0 (score: 4,47)
  3. Doc 2 (score: 3,54)
  4. Doc 1 (score: 2,98)
  5. Doc 5 (score: 2,62)
  6. Doc 3 (score: 2,24)

=> Le modele combine optimalement les deux sources


### Analyse du classement final

**Classement des documents** :

| Rang | Document | Score latent | Jugement | Clics |
|------|----------|--------------|----------|-------|
| 1 | Doc 4 | 5.58 | 5.0 | 150 |
| 2 | Doc 0 | 4.47 | 4.5 | 120 |
| 3 | Doc 2 | 3.54 | 4.0 | 95 |
| 4 | Doc 1 | 2.98 | 3.0 | 80 |
| 5 | Doc 5 | 2.62 | 3.5 | 70 |
| 6 | Doc 3 | 2.24 | 2.5 | 60 |

**Valeur de la fusion** :

Le modèle résout les désaccords entre sources :
- **Doc 5** : Juge=3.5, Clics=70 → Score=2.62 (clics plus bas que attendu → baisse du score)
- **Doc 2** : Juge=4.0, Clics=95 → Score=3.54 (clics cohérents → score intermédiaire)

**Avantages du Click Model** :

| Aspect | Bénéfice |
|--------|----------|
| **Calibration automatique** | Apprend l'échelle de chaque source |
| **Pondération adaptative** | Sources plus précises ont plus de poids |
| **Incertitude explicite** | Quantifie la confiance du classement |
| **Extensible** | Ajouter d'autres signaux (temps lecture, rebonds, etc.) |

## 7. Exercice : Recommandation de Films

### Enonce

Utilisez le modele de factorisation pour predire les notes manquantes et recommander des films.

### Contexte de l'exercice

Cet exercice applique le modele de factorisation a un scenario concret : recommander des **films** a des utilisateurs.

**Mapping semantique** :
| ID | Film | Type suppose |
|----|------|--------------|
| 0 | Inception | Action/Sci-Fi |
| 1 | Titanic | Romance |
| 2 | Matrix | Action/Sci-Fi |
| 3 | Notebook | Romance |
| 4 | Terminator | Action |

**Profils utilisateurs (implicites)** :
- Alice, Claire : Preferences action
- Bob : Preferences romance
- David : Preferences action

### Instructions pour l'exercice

L'exercice utilise le modele de factorisation de la section 3 pour predire des notes sur un jeu de donnees semantique (films).

**Objectif** : Comprendre la relation entre :
1. Les traits latents appris (ou non appris)
2. La qualite des recommandations

**Ce que vous devriez observer** :
- Avec le modele original (8 observations), les recommandations sont uniformes (scores ~0)
- Cela illustre l'importance de la qualite de l'inference en amont

**Extension possible** : Modifier le code pour utiliser les posterieurs du modele corrige (section 3bis) et observer la difference.

In [22]:
// EXERCICE : Recommandation de films

// Films : 0=Inception, 1=Titanic, 2=Matrix, 3=NotebookFilm, 4=Terminator
string[] films = { "Inception", "Titanic", "Matrix", "Notebook", "Terminator" };

// Utilisateurs : 0=Alice, 1=Bob, 2=Claire, 3=David
string[] users = { "Alice", "Bob", "Claire", "David" };

// Notes observees (memes donnees que section 3)
Console.WriteLine("=== Recommandation de Films ===");
Console.WriteLine("\nNotes observees :");
Console.WriteLine($"  Alice -> Inception: 5, Matrix: 3");
Console.WriteLine($"  Bob -> Titanic: 4, Notebook: 2");
Console.WriteLine($"  Claire -> Inception: 4, Matrix: 5");
Console.WriteLine($"  David -> Matrix: 4, Terminator: 5");

// Utiliser les posterieurs calcules precedemment
Console.WriteLine("\n=== Top 3 Recommandations par utilisateur ===");

for (int u = 0; u < nUsers; u++)
{
    var predictions = new List<(int item, double score)>();
    
    for (int i = 0; i < nItems; i++)
    {
        // Verifier si deja note
        bool observe = false;
        for (int o = 0; o < nObs; o++)
        {
            if (userObs[o] == u && itemObs[o] == i)
            {
                observe = true;
                break;
            }
        }
        
        if (!observe)
        {
            double pred = 0;
            for (int t = 0; t < nTraits; t++)
            {
                pred += userTraitsPost[u, t].GetMean() * itemTraitsPost[i, t].GetMean();
            }
            predictions.Add((i, pred));
        }
    }
    
    var top3 = predictions.OrderByDescending(p => p.score).Take(3);
    Console.WriteLine($"\n{users[u]} :");
    foreach (var rec in top3)
    {
        Console.WriteLine($"  -> {films[rec.item]} (score predit: {rec.score:F2})");
    }
}

=== Recommandation de Films ===

Notes observees :
  Alice -> Inception: 5, Matrix: 3
  Bob -> Titanic: 4, Notebook: 2
  Claire -> Inception: 4, Matrix: 5
  David -> Matrix: 4, Terminator: 5

=== Top 3 Recommandations par utilisateur ===

Alice :
  -> Titanic (score predit: 0,00)
  -> Notebook (score predit: -0,00)
  -> Terminator (score predit: -0,00)

Bob :
  -> Terminator (score predit: 0,00)
  -> Matrix (score predit: -0,00)
  -> Inception (score predit: -0,00)

Claire :
  -> Titanic (score predit: 0,00)
  -> Notebook (score predit: -0,00)
  -> Terminator (score predit: -0,00)

David :
  -> Notebook (score predit: 0,00)
  -> Inception (score predit: -0,00)
  -> Titanic (score predit: -0,00)


### Analyse des recommandations de films

**Résultats** : Toutes les recommandations ont un score ~0.0

| Utilisateur | Top recommandations |
|-------------|---------------------|
| Alice | Titanic, Notebook, Terminator (tous ~0.0) |
| Bob | Terminator, Matrix, Inception (tous ~0.0) |
| Claire | Titanic, Notebook, Terminator (tous ~0.0) |
| David | Notebook, Inception, Titanic (tous ~0.0) |

**Diagnostic** :

Les scores nuls reflètent le problème de convergence de la factorisation (voir analyse précédente) :
- Traits utilisateurs ≈ 0 → produit scalaire ≈ 0
- Aucune différenciation possible entre items

**Ce qu'un système fonctionnel montrerait** :

| Utilisateur | Profil attendu | Recommandations attendues |
|-------------|----------------|---------------------------|
| Alice | Action (5 Inception, 3 Matrix) | Terminator (action) |
| Bob | Romance (4 Titanic, 2 Notebook) | Notebook supplémentaire |
| Claire | Action (4 Inception, 5 Matrix) | Terminator |
| David | Action (4 Matrix, 5 Terminator) | Inception |

**Leçon** : La factorisation bayésienne est puissante mais nécessite :
- Suffisamment de données (>10× paramètres)
- Bonne initialisation ou priors informatifs
- Validation des sorties avant déploiement

### Points cles de l'exercice

**Observations principales** :

1. **La qualite des recommandations depend de l'inference** : Un modele mal converge produit des recommandations inutilisables
2. **Diagnostic rapide** : Des scores uniformes signalent un probleme en amont
3. **Solution** : Verifier les traits latents avant de deployer

**Checklist de validation d'un systeme de recommandation** :

| Verification | Methode |
|--------------|---------|
| Traits non-nuls | Inspecter moyennes des posterieurs |
| Variance raisonnable | Traits avec incertitude, pas collapses a 0 |
| Predictions variees | Distribution des scores, pas uniformes |
| Coherence semantique | Top recommendations correspondent aux preferences observees |

> **Conseil pratique** : Toujours valider sur un ensemble de test (notes observees masquees) avant deploiement.

### Bilan : Systemes de recommandation bayesiens

| Modele | Usage | Points cles |
|--------|-------|-------------|
| **Factorisation matricielle** | Prediction de notes | Traits latents, produit scalaire, regularisation par priors |
| **Cold-start hybride** | Nouveaux utilisateurs/items | Features + factorisation, prediction immediate |
| **Click Model** | Fusion de sources | Score latent, calibration automatique, incertitude par source |

**Quand utiliser chaque approche** :

| Situation | Recommandation |
|-----------|----------------|
| Beaucoup de notes, peu de nouveautes | Factorisation pure |
| Forte rotation utilisateurs/items | Cold-start avec features |
| Multiple signaux (clics, temps, notes) | Click Model ou variantes |
| Production a grande echelle | Matchbox (Infer.NET industriel) |

## 8. Resume

| Concept | Description |
|---------|-------------|
| **Factorisation** | Decomposition U x V des preferences latentes |
| **Cold-start** | Utiliser features pour nouveaux utilisateurs/items |
| **Click Model** | Reconcilier sources de qualite variable |
| **Matchbox** | Implementation industrielle dans Infer.NET |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|--------------|
| Debugger des problemes de convergence | [Infer-13-Debugging](Infer-13-Debugging.ipynb) |
| Comprendre les algorithmes EP vs VMP | [Infer-13-Debugging](Infer-13-Debugging.ipynb) Section 4 |
| Ameliorer le ratio donnees/parametres | [Infer-13-Debugging](Infer-13-Debugging.ipynb) Section 2 |
| Trouver une definition | [Glossaire](Infer-Glossary.md) |

---

## Serie Complete

Felicitations ! Vous avez termine la serie **Programmation Probabiliste avec Infer.NET** (13 notebooks).

| # | Notebook | Concepts |
|---|----------|----------|
| 1 | Setup | Installation, premier modele, troubleshooting |
| 2 | Gaussian-Mixtures | Posterieurs, melanges, Truncated Gaussian |
| 3 | Factor-Graphs | Inference discrete, Monty Hall |
| 4 | Bayesian-Networks | CPT, causalite |
| 5 | Skills-IRT | IRT, DINA, many-to-many, ROC curves |
| 6 | TrueSkill | Ranking, online learning |
| 7 | Classification | BPM, A/B testing |
| 8 | Model-Selection | Evidence, ARD |
| 9 | Topic-Models | LDA, documents-topics, brisure de symetrie |
| 10 | Crowdsourcing | Workers, communautes |
| 11 | Sequences | HMM, transitions Markov, motif finding |
| 12 | Recommenders | Factorisation, Click Model |
| **13** | **Debugging** | **Troubleshooting, comparaison algorithmes** |

---

## Ressources

- [Documentation Infer.NET](https://dotnet.github.io/infer/)
- [Livre MBML](https://mbmlbook.com/)
- [Code source Infer.NET](https://github.com/dotnet/infer)
- [Glossaire](Infer-Glossary.md)

## Tableau recapitulatif des distributions utilisees

| Distribution | Usage dans ce notebook | Parametres |
|--------------|------------------------|------------|
| **Gaussian(mean, precision)** | Traits latents, biais global, scores | mean=0/3, precision=0.1-1 |
| **Gamma(shape, scale)** | Precision du bruit | shape=2, scale=0.5 |
| **GaussianFromMeanAndPrecision** | Generation des notes | mean=affinite, precision=bruit |

## Concepts probabilistes illustres

| Concept | Section | Description |
|---------|---------|-------------|
| **Factorisation matricielle** | 3 | Decomposition R = U x V^T avec priors bayesiens |
| **Probleme de convergence** | 3-4 | Ratio donnees/parametres insuffisant |
| **Cold-start** | 5 | Prediction sans historique via features |
| **Fusion multi-sources** | 6 | Click Model avec calibration automatique |
| **Produit de gaussiennes** | 3, 6 | Necessite EP (pas VMP) |

## Applications pratiques

| Domaine | Application | Modele recommande |
|---------|-------------|-------------------|
| **E-commerce** | Recommandation produits | Factorisation + cold-start |
| **Streaming** | Recommandation films/series | Factorisation matricielle |
| **Moteur de recherche** | Ranking documents | Click Model |
| **Reseau social** | Suggestion d'amis | Factorisation avec features sociales |